In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('ch6_dataset.csv')

In [4]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149.0,1.0,9.0,2018-10-19,0.21,6.0,365.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1.0,45.0,2019-05-21,0.38,2.0,355.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150.0,3.0,0.0,NaN,NaN,1.0,365.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89.0,1.0,270.0,2019-07-05,4.64,1.0,194.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80.0,10.0,9.0,2018-11-19,0.10,1.0,0.0


In [27]:
print(df.shape)
df.isnull().sum()

(36334, 16)


id                                   0
name                                16
host_id                              0
host_name                           21
neighbourhood_group                  1
neighbourhood                        1
latitude                             1
longitude                            1
room_type                            1
price                                1
minimum_nights                       1
number_of_reviews                    1
last_review                       5802
reviews_per_month                    0
calculated_host_listings_count       1
availability_365                     1
dtype: int64

In [28]:
#fill missing cells in reviews_per_month with 0s
df.reviews_per_month = df.reviews_per_month.fillna(0)
df.price = df.price.fillna(0)

In [6]:
#Apply the log transform to price
#Do train/validation/test split with 60%/20%/20% distribution.
#Use the train_test_split function and set the random_state parameter to 1

In [29]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = np.log1p(df_train.price.values)
y_val = np.log1p(df_val.price.values)
y_test = np.log1p(df_test.price.values)

del df_train['price']
del df_val['price']
del df_test['price']

In [30]:
#Now, use DictVectorizer to turn train and validation into matrices:
from sklearn.feature_extraction import DictVectorizer

In [31]:
features = [
    'neighbourhood_group', 'room_type', 'latitude', 'longitude',
    'minimum_nights', 'number_of_reviews','reviews_per_month',
    'calculated_host_listings_count', 'availability_365',
]

In [32]:

train_dicts = df_train[features].to_dict(orient='records')
val_dicts = df_val[features].to_dict(orient='records')

In [33]:
dv = DictVectorizer(sparse=False)

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

# Decision Tree Regressor to predict price, use max_depth=1
### Note that we did DecisionTreeClassifier not DecisionTreeRegressor
### The difference is Classifer is for logistic regression
### While Regressor is for linear regression
### Our dependent variable is "price" so it's a linear regression.

In [34]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_text

In [35]:
#to train decision tree
dt = DecisionTreeRegressor(max_depth=1)   #note we did not do this in class, we did DecisionTreeClassifier
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [36]:
print(export_text(dt, feature_names=dv.get_feature_names()))

|--- room_type=Entire home/apt <= 0.50
|   |--- value: [4.28]
|--- room_type=Entire home/apt >  0.50
|   |--- value: [5.12]



In [37]:
#Which feature is used to split the data? Room type

# Random Forest with these parameters
### n_estimators=10
### random_State=1
### n_jobs=-1 (makes training faster)

### Note that we did RandomForestClassifier not RandomForestRegressor
### The difference is Classifer is for logistic regression
### While Regressor is for linear regression.
### Our dependent variable is "price" so it's a linear regression.

In [42]:
from sklearn.ensemble import RandomForestRegressor   
#again we didn't do this in class, we used Classifer

In [43]:
#to train the model
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [44]:
#Were not taught this
# What is RMSE of this model on validation?
from sklearn.metrics import mean_squared_error

In [41]:
y_pred = rf.predict(X_val)
np.sqrt(mean_squared_error(y_val, y_pred))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [45]:
#I can't get this rmse to work!

In [ ]:
#The rest of the homework goes on to use more stuff we didn't learn. So I stopped. This is so frustrating.